# Logiques Avancées - DL, Modale, QBF, Conditional

**Navigation**: [← Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-4-Belief-Revision →](Tweety-4-Belief-Revision.ipynb)

---

## Objectifs pédagogiques

1. Comprendre les logiques de description (DL) et leur lien avec les ontologies
2. Explorer la logique modale avec les opérateurs de nécessité et possibilité
3. Découvrir la logique QBF (Quantified Boolean Formulas)
4. Appréhender la logique conditionnelle (CL)

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - `SimpleMlReasoner` peut bloquer indéfiniment sans solveur SPASS configuré
> - Le parsing de logique modale fonctionne, mais le raisonnement nécessite SPASS
> - QBF et CL sont présentés en aperçu (fonctionnalités de base)

In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE, pas l'executable
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover) - executable complet
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
        else:
            print(f"  {tool}: non configure")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

### 2.3 Logique de Description (DL)
<a id="2.3"></a>

Les logiques de description sont une famille de formalismes pour représenter des connaissances structurées, souvent utilisées pour les ontologies (ex: OWL). Elles se concentrent sur la définition de **Concepts** (classes d'individus), de **Rôles** (relations binaires) et d'**Individus**.

* **Signature (`DlSignature`)** : Comprend `AtomicConcept`, `AtomicRole`, `Individual`.
* **Axiomes** :
    * **TBox (Terminological Box)** : Axiomes définissant les concepts et les rôles (ex: `SubConceptAxiom`, `EquivalenceAxiom`, `DisjointAxiom`). Les concepts peuvent être combinés (`Intersection`, `Union`, `Complement`, `ExistsRestriction`, `ForAllRestriction`).
    * **ABox (Assertional Box)** : Axiomes sur les individus (ex: `ConceptAssertion` - `Human(Alice)`, `RoleAssertion` - `fatherOf(Bob, Alice)`).
* **Base de Connaissances (`DlBeliefSet`)** : Contient les axiomes TBox et ABox.
* **Raisonnement (`DlReasoner`)** : Vérifie la consistance, la subsomption de concepts, l'instanciation. `NaiveDlReasoner` est une implémentation simple. Des raisonneurs plus puissants (comme Pellet, HermiT - non intégrés directement comme solveurs externes dans cet exemple) existent.

In [ ]:
# --- 2.3 Logique de Description ---
print("\n--- 2.3 Logique de Description ---")

# Verifier si la JVM est prete
if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
else:
    print("JVM prete. Execution de l'exemple DL...")
    try:
        # Imports DL et Communs
        import jpype
        from jpype.types import *
        from java.util import ArrayList # Besoin pour Union

        # Classes DL specifiques
        from org.tweetyproject.logics.dl.syntax import (
            AtomicConcept, AtomicRole, Individual,
            EquivalenceAxiom, ConceptAssertion, RoleAssertion,
            DlBeliefSet, DlSignature, DlAxiom,
            Complement, Union
        )
        from org.tweetyproject.logics.dl.parser import DlParser
        from org.tweetyproject.logics.dl.reasoner import NaiveDlReasoner

        print("OK Imports DL reussis.")

        # --- Signature et Axiomes ---
        # Concepts
        human = AtomicConcept("Human")
        male = AtomicConcept("Male")
        female = AtomicConcept("Female")
        house = AtomicConcept("House")
        father = AtomicConcept("Father")
        # Roles
        fatherOf = AtomicRole("fatherOf")
        # Individus
        bob = Individual("Bob")
        alice = Individual("Alice")

        # Axiomes Terminologiques (TBox)
        femaleHuman = EquivalenceAxiom(female, human)
        maleHuman = EquivalenceAxiom(male, human)
        femaleNotMale = EquivalenceAxiom(female, Complement(male))
        maleNotFemale = EquivalenceAxiom(male, Complement(female))

        print("Note: Definition de 'Father' simplifiee pour cet exemple.")

        houseNotHuman = EquivalenceAxiom(house, Complement(human))

        # Axiomes Assertoriels (ABox)
        aliceHuman = ConceptAssertion(alice, human)
        bobHuman = ConceptAssertion(bob, human)
        aliceFemale = ConceptAssertion(alice, female)
        bobMale = ConceptAssertion(bob, male)
        bobFatherOfAlice = RoleAssertion(bob, alice, fatherOf)

        # Base de connaissances
        dbs = DlBeliefSet()
        dbs.add(femaleHuman); dbs.add(maleHuman); dbs.add(femaleNotMale); dbs.add(maleNotFemale)
        dbs.add(houseNotHuman)
        dbs.add(aliceHuman); dbs.add(bobHuman); dbs.add(aliceFemale); dbs.add(bobMale)
        dbs.add(bobFatherOfAlice)

        print("\nDL Knowledge Base (creee manuellement):\n", dbs)
        print("ABox seule:", dbs.getABox())
        print("TBox seule:", dbs.getTBox())

        # --- Raisonnement Naif ---
        print("\nRaisonnement DL (Naif):")
        dbs_reason = DlBeliefSet()
        tweety = Individual("Tweety")
        tweetyMale = ConceptAssertion(tweety, male)
        tweetyHuman = ConceptAssertion(tweety, human)
        dbs_reason.add(aliceFemale)
        dbs_reason.add(tweetyMale)
        dbs_reason.add(maleNotFemale)
        dbs_reason.add(aliceHuman)
        dbs_reason.add(femaleHuman)

        reasoner_dl = NaiveDlReasoner()

        # Query 1: Est-ce que Female implique Human ?
        q1_dl = femaleHuman
        print(f" - Query '{q1_dl}'? {reasoner_dl.query(dbs_reason, q1_dl)}")

        # Query 2: Est-ce que Tweety est Humain ?
        q2_dl = tweetyHuman
        dbs_reason.add(maleHuman)
        print(f" - Query '{q2_dl}' (apres ajout Male=Human)? {reasoner_dl.query(dbs_reason, q2_dl)}")

        # Query 3: Est-ce que Alice est Male ?
        q3_dl = ConceptAssertion(alice, male)
        print(f" - Query '{q3_dl}'? {reasoner_dl.query(dbs_reason, q3_dl)}")


    except ImportError as e:
        print(f"ERREUR d'import pour la Logique de Description : {e}. Verifiez le JAR logics.dl.")
    except jpype.JException as e_java:
        print(f"ERREUR Java generale dans l'exemple DL: {e_java.message()}")
        print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"ERREUR Python inattendue dans l'exemple DL: {e_gen}")
        import traceback
        traceback.print_exc()

### 2.4 Logique Modale (ML)
<a id="2.4"></a>

La logique modale ajoute des opérateurs pour qualifier la *manière* dont une proposition est vraie. Les opérateurs classiques sont :
* `[]` (Box) : Nécessité ("il est nécessaire que", "dans tous les mondes accessibles")
* `<>` (Diamond) : Possibilité ("il est possible que", "dans au moins un monde accessible")

Tweety permet de définir des bases de croyances modales et d'utiliser des raisonneurs.

* **Syntaxe**: Utilise la syntaxe FOL avec les opérateurs `[]` et `<>`.
* **Parsing (`MlParser`)**: Peut parser des formules et bases de croyances modales.
* **Raisonnement**:
    * `SimpleMlReasoner` : Implémentation basique.
    * `SPASSMlReasoner` : Interface avec le prouveur externe SPASS (performant mais nécessite installation et configuration du chemin).

In [ ]:
# --- 2.4 Logique Modale ---
print("\n--- 2.4 Logique Modale ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
else:
    print("JVM prete. Execution de l'exemple ML...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        import pathlib
        IOException = jpype.JClass("java.io.IOException")
        RuntimeException = jpype.JClass("java.lang.RuntimeException")

        from org.tweetyproject.logics.ml.syntax import MlBeliefSet
        from org.tweetyproject.logics.ml.parser import MlParser
        from org.tweetyproject.logics.fol.syntax import FolSignature, FolFormula
        from org.tweetyproject.logics.commons.syntax import Predicate
        from org.tweetyproject.logics.ml.reasoner import AbstractMlReasoner, SimpleMlReasoner, SPASSMlReasoner

        FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
        print("Imports ML reussis.")

        # --- Signature et Base de Croyances ---
        sig_ml = FolSignature()
        p = Predicate("p", 0); q = Predicate("q", 0); r = Predicate("r", 0)
        sig_ml.add(p); sig_ml.add(q); sig_ml.add(r)
        parser_ml = MlParser()
        parser_ml.setSignature(sig_ml)
        kb_ml = MlBeliefSet()
        
        # Syntaxe Modale Tweety:
        #   []  = Box (necessite)
        #   <>  = Diamond (possible)
        formulas_ml = [
            "!(<>(p))",           # NOT possible p
            "p || r",             # p OR r (classique)
            "!r || [](q && r)",   # NOT r OR necessarily(q AND r)
            "[](r && <>(p || q))", # necessarily(r AND possibly(p OR q))
            "!p && !q"            # NOT p AND NOT q
        ]
        
        print("\nParsing des formules ML:")
        parsing_ml_ok = True
        for f_str in formulas_ml:
            try:
                kb_ml.add(parser_ml.parseFormula(f_str))
                print(f"  OK: {f_str}")
            except Exception as e_parse_ml:
                print(f"  ERREUR parsing '{f_str}': {e_parse_ml}")
                parsing_ml_ok = False

        if not parsing_ml_ok:
            print("WARN: Erreurs lors du parsing ML.")
        print(f"\nKB Modale ({kb_ml.size()} formules):")
        print(f"  {kb_ml}")

        # --- Raisonnement ---
        if parsing_ml_ok and kb_ml.size() > 0:
            ml_reasoner = None
            
            # Utiliser SPASS si configure (defini dans la cellule d'init)
            spass_path = get_tool_path('SPASS') if 'get_tool_path' in globals() else None
            
            if spass_path:
                print(f"\nSPASS detecte: {spass_path}")
                try:
                    spass_reasoner = SPASSMlReasoner(JString(spass_path))
                    AbstractMlReasoner.setDefaultReasoner(spass_reasoner)
                    ml_reasoner = AbstractMlReasoner.getDefaultReasoner()
                    print("  SPASS configure comme raisonneur ML par defaut.")
                except Exception as e_spass:
                    print(f"  ERREUR configuration SPASS: {e_spass}")
                    ml_reasoner = None

            if ml_reasoner:
                current_ml_reasoner = AbstractMlReasoner.getDefaultReasoner()
                print(f"\nRaisonneur ML: {current_ml_reasoner.getClass().getSimpleName()}")

                queries_ml_str = ["[](!p)", "<>(q || r)", "p", "r", "[](q)"]
                print("\nResultats des requetes:")
                for query_str in queries_ml_str:
                    print(f"  Query '{query_str}'...", end="")
                    try:
                        query_formula = parser_ml.parseFormula(query_str)
                        result = current_ml_reasoner.query(kb_ml, JObject(query_formula, FolFormula_class))
                        print(f" {result}")
                    except jpype.JException as e_query_java:
                        if "error=740" in str(e_query_java.message()):
                            print(" ERREUR 740 (Elevation requise pour SPASS)")
                        else:
                            print(f" ERREUR: {e_query_java.message()}")
                    except Exception as e_query_py:
                        print(f" ERREUR: {e_query_py}")
            else:
                # SimpleMlReasoner bloque indefiniment - on le documente mais on ne l'utilise pas
                print("\n" + "="*60)
                print("LIMITATION CONNUE: SimpleMlReasoner bloque indefiniment")
                print("="*60)
                print("  Le raisonneur SimpleMlReasoner de Tweety tente d'enumerer")
                print("  tous les mondes possibles modaux, ce qui explose combinatoirement.")
                print("")
                print("  Solution: Installer SPASS (prouveur de theoremes automatique)")
                print("    1. Telecharger depuis: https://www.spass-prover.org/")
                print("    2. Placer l'executable dans ext_tools/spass/SPASS.exe")
                print("    3. Re-executer cette cellule")
                print("")
                print("  Le parsing et la construction de KB fonctionnent correctement.")
                print("  Seul le raisonnement necessite SPASS.")
        else:
            print("\nWARN: Raisonnement ML saute (parsing echoue ou KB vide).")

    except ImportError as e:
        print(f"ERREUR d'import pour la Logique Modale : {e}")
    except jpype.JException as e_java:
        print(f"ERREUR Java generale dans l'exemple ML: {e_java.message()}")
        print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"ERREUR Python inattendue dans l'exemple ML: {e_gen}")
        import traceback
        traceback.print_exc()

### 2.5 Autres Logiques (Aperçu QBF, CL)
<a id="2.5"></a>

Tweety supporte d'autres logiques que nous n'explorerons pas en détail ici, mais dont voici un aperçu fonctionnel :

* **Formules Booléennes Quantifiées (QBF)** (`org.tweetyproject.logics.qbf`): Étend PL avec des quantificateurs existentiels (`exists`) et universels (`forall`) sur les **propositions**. Utile pour modéliser des problèmes PSPACE. Tweety inclut des parseurs pour les formats QDIMACS et QCir, ainsi qu'un writer QDIMACS. Le raisonnement QBF nécessite généralement des solveurs externes spécialisés (non intégrés directement comme `CmdLineQbfSolver` pour le moment, contrairement aux SAT solvers).

* **Logique Conditionnelle (CL)** (`org.tweetyproject.logics.cl`): Traite des conditionnels de la forme `(B | A)` signifiant "Si A est vrai, alors B est typiquement vrai". Utilisée pour le raisonnement non monotone et la modélisation des défauts. Tweety fournit des raisonneurs basés sur les fonctions de classement (`SimpleCReasoner`, `RuleBasedCReasoner`).

In [ ]:
# --- 2.5 Autres Logiques (Apercu QBF et CL) ---
print("\n--- 2.5 Autres Logiques (Apercu QBF et CL) ---")

# Verifier si la JVM est prete
if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
else:
    print("JVM prete. Execution des exemples QBF et CL...")

    # --- QBF ---
    print("\n--- Apercu QBF ---")
    try:
        # Imports QBF et PL
        import jpype
        from jpype.types import *
        from org.tweetyproject.logics.pl.syntax import Proposition, PlBeliefSet, Conjunction, Negation

        # Charger ExistsQuantifiedFormula via JClass ou import
        ExistsQuantifiedFormula = None
        try:
             ExistsQuantifiedFormula = jpype.JClass("org.tweetyproject.logics.qbf.syntax.ExistsQuantifiedFormula")
             print("OK Classe ExistsQuantifiedFormula chargee.")
        except Exception as e_qbf_jclass:
             print(f"WARN Impossible de charger ExistsQuantifiedFormula via JClass: {e_qbf_jclass}. Tentative d'import direct...")
             try:
                  from org.tweetyproject.logics.qbf.syntax import ExistsQuantifiedFormula
                  print("OK Import direct de ExistsQuantifiedFormula reussi.")
             except ImportError:
                   print("ERREUR Import direct de ExistsQuantifiedFormula echoue aussi. Section QBF inutilisable.")
                   ExistsQuantifiedFormula = None

        # Parser/Writer QBF
        from org.tweetyproject.logics.qbf.parser import QbfParser, QdimacsParser, QCirParser
        from org.tweetyproject.logics.qbf.writer import QdimacsWriter

        if ExistsQuantifiedFormula:
            # Creation programmatique de la formule: exists P: (P && !P)
            p_qbf = Proposition("P")
            body_formula = Conjunction(p_qbf, Negation(p_qbf))
            qbf_formula = ExistsQuantifiedFormula(body_formula, p_qbf)

            qbf_bs = PlBeliefSet()
            qbf_bs.add(qbf_formula)
            print("\nQBF BeliefSet (cree programmatiquement):\n", qbf_bs)

            # Conversion en QDIMACS
            print("\nFormat QDIMACS:")
            qdimacs_writer = QdimacsWriter()
            print(qdimacs_writer.printBase(qbf_bs))

            print("\nNote: Parsers QBF, QDIMACS et QCir existent mais necessitent fichiers specifiques.")

    except ImportError as e:
        print(f"ERREUR d'import pour QBF : {e}. Verifiez les JARs logics.qbf et logics.pl.")
    except jpype.JException as e_java:
        print(f"ERREUR Java generale dans l'exemple QBF: {e_java.message()}")
    except Exception as e_gen:
        print(f"ERREUR Python inattendue dans l'exemple QBF: {e_gen}")
        import traceback
        traceback.print_exc()


    # --- Logique Conditionnelle (CL) ---
    print("\n\n--- Apercu Logique Conditionnelle (CL) ---")
    try:
        # Imports CL et PL
        import jpype
        from jpype.types import *
        from org.tweetyproject.logics.cl.syntax import ClBeliefSet, Conditional
        from org.tweetyproject.logics.pl.syntax import Proposition, Negation
        from org.tweetyproject.logics.cl.reasoner import SimpleCReasoner
        from org.tweetyproject.logics.cl.semantics import RankingFunction

        print("OK Imports CL reussis.")

        # Creation de la base CL
        f = Proposition("f"); b = Proposition("b"); p = Proposition("p")
        c1 = Conditional(f, b); c2 = Conditional(b, p); c3 = Conditional(Negation(f), p)
        bs_cl = ClBeliefSet(); bs_cl.add(c1); bs_cl.add(c2); bs_cl.add(c3)
        print("\nBase Conditionnelle:\n", bs_cl)

        # Raisonnement
        cl_reasoner = SimpleCReasoner()
        print(f"\nCalcul de la fonction de classement avec {cl_reasoner.getClass().getSimpleName()}...")
        try:
            kappa_model = cl_reasoner.getModel(bs_cl)
            print("\nFonction de Classement (kappa) calculee:\n")
            print(str(kappa_model))

        except jpype.JException as e_cl_reason:
            print(f"ERREUR Java lors du calcul du modele CL: {e_cl_reason.message()}")

    except ImportError as e:
        print(f"ERREUR d'import pour CL : {e}. Verifiez le JAR logics.cl.")
    except jpype.JException as e_java:
        print(f"ERREUR Java generale dans l'exemple CL: {e_java.message()}")
    except Exception as e_gen:
        print(f"ERREUR Python inattendue dans l'exemple CL: {e_gen}")
        import traceback
        traceback.print_exc()

---

## Resume

Ce notebook a couvert:
- **Description Logic (DL)**: Concepts, roles, ABox/TBox, classification
- **Logique Modale (ML)**: Operateurs Box/Diamond, accessibilite, parsing
- **QBF et Conditional Logic**: Apercu des formalismes etendus

**Limitations connues:**
- `SimpleMlReasoner` bloque indefiniment (enumeration combinatoire) - utiliser SPASS
- Les raisonneurs QBF/CL peuvent necessiter des solveurs externes

**Outils externes utiles:**
| Outil | Usage | Installation |
|-------|-------|--------------|
| SPASS | Raisonnement modal | [spass-prover.org](https://www.spass-prover.org/) |
| EProver | Raisonnement FOL | [eprover.org](https://eprover.org/) |

## Prochaines etapes

Le notebook suivant aborde la revision de croyances, les mesures d'incoherence et les techniques MUS/MaxSAT.

---

**Navigation**: [<- Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-4-Belief-Revision ->](Tweety-4-Belief-Revision.ipynb)